In [0]:
# Databricks notebook cell (Python 3.10+)
# DEMO: "Databricks Genie" Multi-Agent Supervisor (Agent Bricks) — minimal working example
# What this cell does:
#   1) Auth via databricks-sdk
#   2) (Optional) Look up your Genie Space to verify access
#   3) Define your already-created Agent Bricks "Multi-Agent Supervisor" endpoint name
#   4) Provide a simple chat() helper to invoke the supervisor like a normal serving endpoint
#
# Prereqs:
#   - A Genie Space exists (note its name or ID)
#   - A Multi-Agent Supervisor endpoint is created in Agent Bricks UI that includes:
#       • sub-agent: your Genie Space (for SQL/structured)
#       • sub-agent: (optional) a Knowledge Assistant (RAG over docs)
#       • (optional) UC Function tools or an MCP server
#   - You have CAN QUERY on the supervisor endpoint and access to included sub-agents.
#   - DATABRICKS_HOST and DATABRICKS_TOKEN (or OBO OAuth) available to the notebook.

%pip install -q databricks-sdk==0.32.0 requests

import json, os, time, uuid, requests
from databricks.sdk import WorkspaceClient
from databricks.sdk.core import OAuthCredentials
from databricks.sdk.service.dashboards import ListSpacesRequest

# ---------- CONFIG ----------
# REQUIRED: set these to match your workspace & artifacts
DATABRICKS_HOST   = os.environ.get("DATABRICKS_HOST") or dbutils.secrets.get("workspace", "DATABRICKS_HOST")
DATABRICKS_TOKEN  = os.environ.get("DATABRICKS_TOKEN") or dbutils.secrets.get("workspace", "DATABRICKS_TOKEN")

# Name (or ID) of your Genie Space (used here only to sanity-check you can see it)
GENIE_SPACE_NAME  = os.environ.get("GENIE_SPACE_NAME", "Sales Analytics Space")

# Name of the Agent Bricks Multi-Agent Supervisor serving endpoint you created in the UI
SUPERVISOR_ENDPOINT = os.environ.get("SUPERVISOR_ENDPOINT", "mas_supervisor_sales_assist")

# Optional: end_user_id lets the supervisor enforce "on-behalf-of user" auth policies if enabled
END_USER_ID = os.environ.get("END_USER_ID", "demo.user@databricks.local")

# ---------- AUTH ----------
# You can use PAT (simple) or OAuth (preferred for production).
w = WorkspaceClient(host=DATABRICKS_HOST, token=DATABRICKS_TOKEN)

# ---------- (Optional) GENIE sanity-check ----------
# List spaces and confirm the Genie Space exists. (Uses the public Genie API via SDK.)
try:
    spaces = w.genie.list_spaces(ListSpacesRequest(page_size=200)).spaces or []
    match = [s for s in spaces if s.name == GENIE_SPACE_NAME or s.id == GENIE_SPACE_NAME]
    if not match:
        print(f"[WARN] Genie Space '{GENIE_SPACE_NAME}' not found in this workspace. You can still invoke your supervisor.")
    else:
        print(f"[OK] Found Genie Space: name='{match[0].name}' id='{match[0].id}'")
except Exception as e:
    print("[INFO] Could not list Genie spaces via SDK in this context; continuing. Error:", repr(e))

# ---------- INVOCATION HELPERS ----------
def _serving_url(endpoint_name: str) -> str:
    # Standard invocations route for serving endpoints
    base = DATABRICKS_HOST.rstrip("/")
    return f"{base}/serving-endpoints/{endpoint_name}/invocations"

def chat_supervisor(prompt: str,
                    endpoint_name: str = SUPERVISOR_ENDPOINT,
                    end_user_id: str = END_USER_ID,
                    temperature: float = 0.1,
                    max_tokens: int = 1024,
                    stream: bool = False) -> dict:
    """
    Send a chat-style request to the Multi-Agent Supervisor endpoint.

    Tip: The exact schema for your endpoint can be copied from the UI via Playground → Get code.
         This generic 'messages' payload works for chat-style agents.
    """
    url = _serving_url(endpoint_name)
    headers = {
        "Authorization": f"Bearer {DATABRICKS_TOKEN}",
        "Content-Type": "application/json",
        # Optional end-user routing header if your workspace uses on-behalf-of-user authorization:
        "X-Databricks-End-User-Id": end_user_id
    }
    # Generic chat format; the supervisor will decide which sub-agent(s) to call.
    payload = {
        "inputs": [{
            "messages": [
                {"role": "system", "content": "You are a helpful enterprise assistant. If the user asks about tables, prefer the Genie Space to answer with SQL-backed facts. Cite sources when available."},
                {"role": "user",   "content": prompt}
            ],
            "parameters": {
                "temperature": temperature,
                "max_tokens": max_tokens,
                "stream": stream
            }
        }]
    }

    resp = requests.post(url, headers=headers, data=json.dumps(payload), timeout=120)
    resp.raise_for_status()
    return resp.json()

# ---------- DEMO QUERIES ----------
test_prompts = [
    # Routed to Genie (structured): aggregation/join queries over Unity Catalog datasets configured in your Genie Space
    "In the last 90 days, which 5 customers had the highest total revenue? Show a short table.",

    # Routed to RAG agent (unstructured): pulls from Knowledge Assistant sub-agent if you added one
    "Summarize our Q3 sales playbook highlights and cite the source documents.",

    # Tooling: if you attached UC Function tools (e.g., create_ticket()), the supervisor can call them when asked
    "Open a ticket to investigate the decline in conversion for Product X last week; include top 3 hypotheses."
]

for q in test_prompts:
    print("\n", "="*30, "\nUSER:", q)
    try:
        out = chat_supervisor(q)
        # Many Agent Bricks endpoints return a standard 'outputs' list with the assistant message in it.
        # We defensively dig for text content:
        txt = None
        if isinstance(out, dict):
            # Common patterns seen in Playground "Get code" samples:
            if "outputs" in out and out["outputs"]:
                msg = out["outputs"][0]
                # msg might be {'content': '...'} or {'messages': [...]} depending on the endpoint schema
                if isinstance(msg, dict):
                    if "content" in msg and isinstance(msg["content"], str):
                        txt = msg["content"]
                    elif "messages" in msg and msg["messages"]:
                        # take the last assistant message content
                        for m in reversed(msg["messages"]):
                            if m.get("role") == "assistant":
                                txt = m.get("content")
                                break
        print("ASSISTANT:", (txt or json.dumps(out)[:2000]))
    except requests.HTTPError as he:
        print("[HTTP ERROR]", he.response.status_code, he.response.text[:500])
    except Exception as e:
        print("[ERROR]", repr(e))

print("\nDone. You can now call chat_supervisor('your question') interactively.")
